In [78]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader, AzureAIDocumentIntelligenceLoader, PyMuPDFLoader, PyPDFLoader
from langchain_community.docstore.document import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from bs4 import BeautifulSoup
from langchain_community.vectorstores import Qdrant
from langchain.retrievers import ParentDocumentRetriever, ContextualCompressionRetriever, SelfQueryRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter, DocumentCompressorPipeline
from langchain.storage import InMemoryStore
from langchain_openai import OpenAI, ChatOpenAI
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import re
import os
import openai
from getpass import getpass
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from ragas.testset.generator import TestsetGenerator
import os
from ragas.testset.evolutions import simple, reasoning, multi_context

In [4]:
openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [62]:
loader = PyPDFLoader("meta_10k.pdf")
documents = loader.load()

In [63]:
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=100
)
texts = text_splitter.split_documents(documents)

In [64]:
len(texts)

147

In [65]:
embeddings_model = OpenAIEmbeddings(model = "text-embedding-3-small")

In [66]:
qdrant = Qdrant.from_documents(texts, embeddings_model, location=":memory:", collection_name = '10K_RAG', force_recreate=True)
qdrant

In [67]:
qdrant_retriever = qdrant.as_retriever()
qdrant_retriever

VectorStoreRetriever(tags=['Qdrant', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.qdrant.Qdrant object at 0x139c7bf50>)

In [68]:
llm = ChatOpenAI(temperature=0)
llm.model_name

'gpt-3.5-turbo'

In [69]:
template = """You are an helpful assistant for question-answering tasks, specifically you are an expert in answering SEC 10-K report questions.
If you  don't know the answer, just say that you don't know.
Answer based on the context given to you, for a given question.

Context:
{context}

Question:
{question}

Answer:"""

rag_chat_prompt = ChatPromptTemplate.from_template(template)
print(rag_chat_prompt.messages[0].prompt.template)

You are an helpful assistant for question-answering tasks, specifically you are an expert in answering SEC 10-K report questions.
If you  don't know the answer, just say that you don't know.
Answer based on the context given to you, for a given question.

Context:
{context}

Question:
{question}

Answer:


In [70]:
base_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_chat_prompt | llm, "context": itemgetter("context")}
)

In [71]:
query1 = "What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"
query2 = "Who are Meta's 'Directors' (i.e., members of the Board of Directors)?"

In [72]:
query1_result = base_chain.invoke({"question": query1})
query2_result = base_chain.invoke({"question": query2})

In [73]:
print(query1_result['response'].content)
print("*******")
print(query2_result['response'].content)

The total value of 'Cash and cash equivalents' as of December 31, 2023, was $41.862 billion.
*******
Meta's Board of Directors includes individuals who oversee strategic and business risk management. The Audit & Risk Oversight Committee of the board is responsible for cybersecurity risk management oversight, while the Privacy Committee oversees privacy and data use risks. Management is responsible for identifying, assessing, and managing material cybersecurity risks. The Chief Information Security Officer (CISO), Guy Rosen, leads the cybersecurity program at Meta.


In [75]:
loader = PyMuPDFLoader("meta_10k.pdf")
documents_eval = loader.load()

In [76]:
text_splitter_eval = RecursiveCharacterTextSplitter(chunk_size = 4250, chunk_overlap = 100)
texts_eval = text_splitter_eval.split_documents(documents_eval)

In [79]:
generator_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo")
embeddings = OpenAIEmbeddings()

generator_test = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

distributions = {
    simple: 0.4,
    multi_context: 0.4,
    reasoning: 0.2
}

testset = generator_test.generate_with_langchain_docs(documents_eval, 12, distributions, is_async = False)
testset.to_pandas()

embedding nodes:   0%|          | 0/346 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/12 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What is the purpose of the facilities consolid...,[Table of Contents\nWe also recorded $1.76 bil...,The purpose of the facilities consolidation re...,simple,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True
1,"What acquisitions did the company make, includ...","[, including by acquiring Instagram in 2012 an...","The company made several acquisitions, includi...",simple,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True
2,What types of claims and lawsuits have been fi...,[ litigation and other proceedings that were f...,Putative class actions have been filed in the ...,simple,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True
3,How are accounts receivable recorded and carri...,[Table of Contents\nidentification method and ...,Accounts receivable are recorded and carried a...,simple,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True
4,What are some areas in which government author...,[ New legislation or regulatory decisions that...,government authorities and regulators are inve...,simple,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True
5,What are the consequences for a company's repu...,[Table of Contents\nPayment-related activities...,"Security breaches, improper access to or discl...",multi_context,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True
6,What is the revenue recognition basis at Meta ...,"[Table of Contents\nMETA PLATFORMS, INC.\nNOTE...","Meta Platforms, Inc. recognizes revenue under ...",multi_context,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True
7,What are the potential impacts on our products...,[Table of Contents\n•\nuser behavior on any of...,The potential impacts on our products and serv...,multi_context,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True
8,How are gains or losses in interest and other ...,[Table of Contents\nidentification method and ...,We include gains or losses in interest and oth...,multi_context,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True
9,How does the conversion of Class B common stoc...,[Table of Contents\nNote 4. Earnings per Share...,The conversion of Class B common stock to Clas...,multi_context,"[{'source': 'meta_10k.pdf', 'file_path': 'meta...",True


In [80]:
test_df = testset.to_pandas()

In [81]:
test_questions = test_df["question"].values.tolist()
test_groundtruths = test_df["ground_truth"].values.tolist()

In [82]:
answers = []
contexts = []

for question in test_questions:
  response = base_chain.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])
  print('Question: ' + question)
  print('Answer: ' + response["response"].content)
  print("****")

Question: What is the purpose of the facilities consolidation restructuring efforts mentioned in the context?
Answer: The purpose of the facilities consolidation restructuring efforts mentioned in the context is to pursue greater efficiency and realign the business and strategic priorities of the company. This includes subleasing, early terminating, or abandoning several office buildings under operating leases, as well as a pivot towards a next-generation data center design.
****
Question: What acquisitions did the company make, including acquiring Instagram?
Answer: The company made acquisitions including acquiring Instagram in 2012 and WhatsApp in 2014.
****
Question: What types of claims and lawsuits have been filed regarding the alleged "social media addiction" caused by Facebook and Instagram?
Answer: Multiple putative class actions have been filed in various federal and state courts alleging that Facebook and Instagram cause "social media addiction" in users, with most proceeding

In [83]:
from datasets import Dataset

response_dataset = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [88]:
response_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 12
})

In [84]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [89]:
results_baseline = evaluate(response_dataset, metrics)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [90]:
results_baseline

{'faithfulness': 1.0000, 'answer_relevancy': 0.9463, 'context_recall': 0.9474, 'context_precision': 0.9606, 'answer_correctness': 0.7568}

In [91]:
results_baseline = results_baseline.to_pandas()
results_baseline

/Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the purpose of the facilities consolid...,The purpose of the facilities consolidation re...,[Table of Contents\nNote 3. Restructuring\n202...,The purpose of the facilities consolidation re...,1.0,1.000000,1.000000,0.805556,0.225313
1,"What acquisitions did the company make, includ...",The company made acquisitions including acquir...,[Table of Contents\nThese laws and regulations...,"The company made several acquisitions, includi...",1.0,0.920075,1.000000,0.916667,0.739880
2,What types of claims and lawsuits have been fi...,Multiple putative class actions have been file...,"[Table of Contents\nthe Sherman Act, including...",Putative class actions have been filed in the ...,1.0,0.938838,1.000000,1.000000,0.810364
3,How are accounts receivable recorded and carri...,Accounts receivable are recorded and carried a...,[Table of Contents\nidentification method and ...,Accounts receivable are recorded and carried a...,1.0,0.983201,1.000000,0.805556,0.740012
4,What are some areas in which government author...,Government authorities and regulators are inve...,[Table of Contents\nIf our new products or cha...,government authorities and regulators are inve...,1.0,0.977872,1.000000,1.000000,0.829834
5,What are the consequences for a company's repu...,The consequences for a company's reputation an...,[Table of Contents\nPayment-related activities...,"Security breaches, improper access to or discl...",1.0,0.950771,1.000000,1.000000,0.860306
6,What is the revenue recognition basis at Meta ...,The revenue recognition basis at Meta Platform...,"[Table of Contents\nMETA PLATFORMS, INC.\nNOTE...","Meta Platforms, Inc. recognizes revenue under ...",1.0,0.963448,1.000000,1.000000,0.648836
7,What are the potential impacts on our products...,The potential impacts on our products and serv...,[Table of Contents\n•third-party initiatives t...,The potential impacts on our products and serv...,1.0,0.889799,0.368421,1.000000,0.918912
8,How are gains or losses in interest and other ...,Gains or losses in interest and other income (...,[Table of Contents\nidentification method and ...,We include gains or losses in interest and oth...,1.0,0.944434,1.000000,1.000000,0.804831
9,How does the conversion of Class B common stoc...,The conversion of Class B common stock to Clas...,[Table of Contents\nNote 4. Earnings per Share...,The conversion of Class B common stock to Clas...,1.0,0.993349,1.000000,1.000000,0.622546


In [103]:
loader = PyMuPDFLoader("meta_10k.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 4250, chunk_overlap = 100)
texts = text_splitter.split_documents(documents)
embeddings_model = OpenAIEmbeddings(model = "text-embedding-3-large")
qdrant = Qdrant.from_documents(texts, embeddings_model, location=":memory:", collection_name = '10K_RAG', force_recreate=True)
qdrant
qdrant_retriever = qdrant.as_retriever()
qdrant_retriever
llm = ChatOpenAI(temperature=0)
llm.model_name
metadata_field_info = [
]

document_content_desc = "Form 10-K annual report required by the U.S. Securities and Exchange Commission (SEC), that gives a comprehensive summary of a company's financial performance for company Meta for year 2023"

self_query_retriever = SelfQueryRetriever.from_llm(llm,
                                                   qdrant, 
                                                   document_content_desc, 
                                                   metadata_field_info)
template = """You are an helpful assistant for question-answering tasks, specifically you are an expert in answering SEC 10-K report questions.
If you  don't know the answer, just say that you don't know.
Answer based on the context given to you, for a given question.

Context:
{context}

Question:
{question}

Answer:"""

rag_chat_prompt = ChatPromptTemplate.from_template(template)
print(rag_chat_prompt.messages[0].prompt.template)
rag_qa_chain_sqr =  (
    {"context": itemgetter("question") | self_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_chat_prompt | llm, "context": itemgetter("context")}
)

You are an helpful assistant for question-answering tasks, specifically you are an expert in answering SEC 10-K report questions.
If you  don't know the answer, just say that you don't know.
Answer based on the context given to you, for a given question.

Context:
{context}

Question:
{question}

Answer:


In [104]:
query1_result = rag_qa_chain_sqr.invoke({"question": query1})
query2_result = rag_qa_chain_sqr.invoke({"question": query2})

In [105]:
print(query1_result['response'].content)
print("*******")
print(query2_result['response'].content)

The total value of 'Cash and cash equivalents' as of December 31, 2023, was $41,862 million.
*******
The directors of Meta (formerly Facebook) as per the SEC 10-K report are:
1. Mark Zuckerberg
2. Susan Li
3. Aaron Anderson
4. Peggy Alford
5. Marc L. Andreessen
6. Andrew W. Houston
7. Nancy Killefer
8. Robert M. Kimmitt
9. Sheryl K. Sandberg
10. Tracey T. Travis
11. Tony Xu


In [106]:
answers = []
contexts = []

for question in test_questions:
  response = rag_qa_chain_sqr.invoke({"question" : question})
  answers.append(response["response"].content)
  contexts.append([context.page_content for context in response["context"]])
  print('Question: ' + question)
  print('Answer: ' + response["response"].content)
  print("****")

Question: What is the purpose of the facilities consolidation restructuring efforts mentioned in the context?
Answer: The purpose of the facilities consolidation restructuring efforts mentioned in the context is to pursue greater efficiency and realign the company's business and strategic priorities. This includes subleasing, early terminating, or abandoning several office buildings under operating leases to streamline operations and reduce costs.
****
Question: What acquisitions did the company make, including acquiring Instagram?
Answer: The company made acquisitions including acquiring Instagram in 2012 and WhatsApp in 2014.
****
Question: What types of claims and lawsuits have been filed regarding the alleged "social media addiction" caused by Facebook and Instagram?
Answer: Multiple types of claims and lawsuits have been filed regarding the alleged "social media addiction" caused by Facebook and Instagram. These include putative class actions, derivative actions, public nuisance c

In [108]:
response_dataset_advanced_retrieval = Dataset.from_dict({
    "question" : test_questions,
    "answer" : answers,
    "contexts" : contexts,
    "ground_truth" : test_groundtruths
})

In [109]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    answer_correctness,
    context_recall,
    context_precision,
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
]

In [99]:
results_advanced= evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

In [100]:
results_advanced 

{'faithfulness': 1.0000, 'answer_relevancy': 0.9551, 'context_recall': 0.9825, 'context_precision': 0.8657, 'answer_correctness': 0.6368}

In [107]:
results_advanced = results_advanced.to_pandas()
results_advanced

/Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the purpose of the facilities consolid...,The purpose of the facilities consolidation re...,[Table of Contents\nNote 3. Restructuring\n202...,The purpose of the facilities consolidation re...,1.0,1.000000,1.000000,0.805556,0.225233
1,"What acquisitions did the company make, includ...",The company made acquisitions of Instagram in ...,"[Europe, and other jurisdictions. Such investi...","The company made several acquisitions, includi...",1.0,0.885400,1.000000,1.000000,0.613656
2,What types of claims and lawsuits have been fi...,Multiple types of claims and lawsuits have bee...,[and certain of our directors and officers all...,Putative class actions have been filed in the ...,1.0,0.999583,1.000000,1.000000,0.596731
3,How are accounts receivable recorded and carri...,Accounts receivable are recorded and carried a...,[other unobservable inputs including volatilit...,Accounts receivable are recorded and carried a...,1.0,0.983201,1.000000,0.583333,0.614254
4,What are some areas in which government author...,Government authorities and regulators are inve...,"[cybersecurity attacks, data privacy violation...",government authorities and regulators are inve...,1.0,0.985153,1.000000,1.000000,0.829877
5,What are the consequences for a company's repu...,Cyber incidents can have significant consequen...,[Table of Contents\nPayment-related activities...,"Security breaches, improper access to or discl...",1.0,0.902844,1.000000,1.000000,0.790593
6,What is the revenue recognition basis at Meta ...,The revenue recognition basis at Meta Platform...,[],"Meta Platforms, Inc. recognizes revenue under ...",1.0,0.955150,1.000000,0.000000,0.653773
7,What are the potential impacts on our products...,The potential impacts on products and services...,[Table of Contents\neffectiveness of advertisi...,The potential impacts on our products and serv...,1.0,0.981549,0.789474,1.000000,0.613446
8,How are gains or losses in interest and other ...,Gains or losses in interest and other income (...,[Table of Contents\nidentification method and ...,We include gains or losses in interest and oth...,1.0,0.949148,1.000000,1.000000,0.742821
9,How does the conversion of Class B common stoc...,The conversion of Class B common stock to Clas...,[Table of Contents\nNote 4. Earnings per Share...,The conversion of Class B common stock to Clas...,1.0,1.000000,1.000000,1.000000,0.622304


In [110]:
results_advanced_large= evaluate(response_dataset_advanced_retrieval, metrics)

Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

No statements were generated from the answer.
Task was destroyed but it is pending!
task: <Task pending name='Task-2275' coro=<as_completed.<locals>.sema_coro() running at /Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/site-packages/ragas/executor.py:38> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/asyncio/futures.py:387, Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/asyncio/tasks.py:602]>
Task was destroyed but it is pending!
task: <Task pending name='Task-2260' coro=<as_completed.<locals>.sema_coro() running at /Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/site-packages/ragas/executor.py:38> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/asyncio/futures.py:387, Task.task_wakeup()]> cb=[as_completed.<locals>._on

In [111]:
results_advanced_large

{'faithfulness': 0.9870, 'answer_relevancy': 0.8886, 'context_recall': 1.0000, 'context_precision': 0.7616, 'answer_correctness': 0.6182}

In [112]:
results_advanced_large = results_advanced_large.to_pandas()
results_advanced_large

/Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/site-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/Users/nithin.kamavaram/anaconda3/envs/AIM/lib/python3.11/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness
0,What is the purpose of the facilities consolid...,The purpose of the facilities consolidation re...,[Table of Contents\nNote 3. Restructuring\n202...,The purpose of the facilities consolidation re...,1.000000,1.000000,1.0,0.833333,0.227668
1,"What acquisitions did the company make, includ...",The company made acquisitions including acquir...,"[Europe, and other jurisdictions. Such investi...","The company made several acquisitions, includi...",1.000000,0.911045,1.0,1.000000,0.739880
2,What types of claims and lawsuits have been fi...,Multiple types of claims and lawsuits have bee...,[and certain of our directors and officers all...,Putative class actions have been filed in the ...,1.000000,0.999583,1.0,0.805556,0.590202
3,How are accounts receivable recorded and carri...,Accounts receivable are recorded and carried a...,[other unobservable inputs including volatilit...,Accounts receivable are recorded and carried a...,1.000000,0.983201,1.0,0.500000,0.743609
4,What are some areas in which government author...,Government authorities and regulators are inve...,"[cybersecurity attacks, data privacy violation...",government authorities and regulators are inve...,1.000000,0.985153,1.0,1.000000,0.830005
5,What are the consequences for a company's repu...,The consequences for a company's reputation an...,[Table of Contents\nPayment-related activities...,"Security breaches, improper access to or discl...",0.857143,0.950774,1.0,1.000000,0.782836
6,What is the revenue recognition basis at Meta ...,The revenue recognition basis at Meta Platform...,[],"Meta Platforms, Inc. recognizes revenue under ...",1.000000,0.955150,1.0,0.000000,0.475201
7,What are the potential impacts on our products...,The potential impacts on products and services...,[Table of Contents\ndevelopments such as the D...,The potential impacts on our products and serv...,1.000000,0.982916,1.0,1.000000,0.712270
8,How are gains or losses in interest and other ...,Gains or losses in interest and other income (...,[Table of Contents\nidentification method and ...,We include gains or losses in interest and oth...,1.000000,0.943551,1.0,1.000000,0.743780
9,How does the conversion of Class B common stoc...,The conversion of Class B common stock to Clas...,[Table of Contents\nNote 4. Earnings per Share...,The conversion of Class B common stock to Clas...,1.000000,0.993823,1.0,1.000000,0.622227


In [114]:
import pandas as pd

In [120]:
# calculating mean for all the metrics
results1 = results_baseline.describe().loc['mean'].reset_index()
results1.columns = ['Metric', 'method1']
results2 = results_advanced.describe().loc['mean'].reset_index()
results2.columns = ['Metric', 'method2']
results3 = results_advanced_large.describe().loc['mean'].reset_index()
results3.columns = ['Metric', 'method3']

In [121]:
# merging all dataframes
df_merged = pd.merge(results1, results2, on='Metric')
df_merged = pd.merge(df_merged, results3, on="Metric")
df_merged

,Metric,method1,method2,method3
0,faithfulness,1.000000,1.000000,0.987013
1,answer_relevancy,0.946302,0.955090,0.888575
2,context_recall,0.947368,0.982456,1.000000
3,context_precision,0.960648,0.865741,0.761574
4,answer_correctness,0.756830,0.636843,0.618194
